In [12]:
import pandas as pd
import matplotlib as mplt
import numpy as np
import seaborn as sb
import sklearn as skl
import yfinance as yf

In [22]:
#https://towardsdatascience.com/analyzing-world-stock-indices-performance-in-python-610df6a578f
market_indices_2023 = []
market_indices_blackTuesday = []
market_indices_blackMonday = []
market_indices_dotcom = []
market_indices_global_crisis = []
market_indices_covid = []

world_indices_page = pd.read_html("https://finance.yahoo.com/world-indices/")
world_indices = world_indices_page[0]

for index in world_indices.Symbol:
  tickerData = yf.Ticker(index)
  tickerDf = tickerData.history(period = "1d", start = "2023-1-1", end = "2023-10-31")
  tickerDf["Index Ticker"] = index
  market_indices_2023.append(tickerDf)

market_indices_2023 = pd.concat(market_indices_2023, axis = 0) #Creating recent index dataframe
market_indices_2023.reset_index(inplace = True) #Fixing formatting, adding Date as a column

^GSPC
^DJI
^IXIC
^NYA
^XAX
^BUK100P
^RUT
^VIX
^FTSE
^GDAXI
^FCHI
^STOXX50E
^N100
^BFX
IMOEX.ME
^N225
^HSI
000001.SS
399001.SZ
^STI
^AXJO
^AORD
^BSESN
^JKSE
^KLSE
^NZ50
^KS11
^TWII
^GSPTSE
^BVSP
^MXX
^IPSA
^IPSA: No data found for this date range, symbol may be delisted
^MERV
^TA125.TA
^CASE30
^JN0U.JO


In [14]:
#Removing extra time data in Date column, help from ChatGPT
market_indices_2023["Date"] = pd.to_datetime(market_indices_2023["Date"], utc = True)
market_indices_2023["Date"] = market_indices_2023["Date"].dt.strftime("%Y-%m-%d")
market_indices_2023

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Index Ticker,Adj Close
0,2023-01-03,3853.290039,3878.459961,3794.330078,3824.139893,3.959140e+09,0.0,0.0,^GSPC,NaN
1,2023-01-04,3840.360107,3873.159912,3815.770020,3852.969971,4.414080e+09,0.0,0.0,^GSPC,NaN
2,2023-01-05,3839.739990,3839.739990,3802.419922,3808.100098,3.893450e+09,0.0,0.0,^GSPC,NaN
3,2023-01-06,3823.370117,3906.189941,3809.560059,3895.080078,3.923560e+09,0.0,0.0,^GSPC,NaN
4,2023-01-09,3910.820068,3950.570068,3890.419922,3892.090088,4.311770e+09,0.0,0.0,^GSPC,NaN
...,...,...,...,...,...,...,...,...,...,...
6748,2023-10-11,3897.689941,3927.010010,3816.580078,3836.889893,0.000000e+00,0.0,0.0,^JN0U.JO,NaN
6749,2023-10-12,3823.189941,3852.129883,3793.110107,3800.600098,0.000000e+00,0.0,0.0,^JN0U.JO,NaN
6750,2023-10-15,3807.610107,3858.659912,3782.330078,3851.270020,0.000000e+00,0.0,0.0,^JN0U.JO,NaN
6751,2023-10-16,3855.219971,3856.739990,3789.149902,3847.870117,0.000000e+00,0.0,0.0,^JN0U.JO,NaN


In [15]:
#Doing EDA on features to see their relevance
open_null = market_indices_2023["Open"].isnull().value_counts()
high_null = market_indices_2023["High"].isnull().value_counts()
low_null = market_indices_2023["Low"].isnull().value_counts()
close_null = market_indices_2023["Close"].isnull().value_counts()
volume_null = market_indices_2023["Volume"].isnull().value_counts()
dividends_null = market_indices_2023["Dividends"].isnull().value_counts()
stockSplits_null = market_indices_2023["Stock Splits"].isnull().value_counts()
index_null = market_indices_2023["Index Ticker"].isnull().value_counts()
adjClose_null = market_indices_2023["Adj Close"].isnull().value_counts()
print(open_null, high_null, low_null, close_null, volume_null, dividends_null, stockSplits_null, index_null, adjClose_null)

Open
False    6753
Name: count, dtype: int64 High
False    6753
Name: count, dtype: int64 Low
False    6753
Name: count, dtype: int64 Close
False    6753
Name: count, dtype: int64 Volume
False    6753
Name: count, dtype: int64 Dividends
False    6753
Name: count, dtype: int64 Stock Splits
False    6753
Name: count, dtype: int64 Index Ticker
False    6753
Name: count, dtype: int64 Adj Close
True    6753
Name: count, dtype: int64


In [16]:
#adj_close gets dropped since an market index does not have such, thus its null in this dataset
market_indices_2023.drop(columns = ["Adj Close"], inplace = True)
market_indices_2023

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Index Ticker
0,2023-01-03,3853.290039,3878.459961,3794.330078,3824.139893,3.959140e+09,0.0,0.0,^GSPC
1,2023-01-04,3840.360107,3873.159912,3815.770020,3852.969971,4.414080e+09,0.0,0.0,^GSPC
2,2023-01-05,3839.739990,3839.739990,3802.419922,3808.100098,3.893450e+09,0.0,0.0,^GSPC
3,2023-01-06,3823.370117,3906.189941,3809.560059,3895.080078,3.923560e+09,0.0,0.0,^GSPC
4,2023-01-09,3910.820068,3950.570068,3890.419922,3892.090088,4.311770e+09,0.0,0.0,^GSPC
...,...,...,...,...,...,...,...,...,...
6748,2023-10-11,3897.689941,3927.010010,3816.580078,3836.889893,0.000000e+00,0.0,0.0,^JN0U.JO
6749,2023-10-12,3823.189941,3852.129883,3793.110107,3800.600098,0.000000e+00,0.0,0.0,^JN0U.JO
6750,2023-10-15,3807.610107,3858.659912,3782.330078,3851.270020,0.000000e+00,0.0,0.0,^JN0U.JO
6751,2023-10-16,3855.219971,3856.739990,3789.149902,3847.870117,0.000000e+00,0.0,0.0,^JN0U.JO


In [17]:
#Checking if all vals in dividends & stock splits are zero (they should be)
dividends_count = market_indices_2023["Dividends"].value_counts()
stockSplits_count = market_indices_2023["Stock Splits"].value_counts()
display(dividends_count, stockSplits_count)

#Since all these vals are zero, we can drop these columns to reduce dimensionality
market_indices_2023.drop(columns = ["Dividends","Stock Splits"], inplace = True)
market_indices_2023

Dividends
0.0    6753
Name: count, dtype: int64

Stock Splits
0.0    6753
Name: count, dtype: int64

,Date,Open,High,Low,Close,Volume,Index Ticker
0,2023-01-03,3853.290039,3878.459961,3794.330078,3824.139893,3.959140e+09,^GSPC
1,2023-01-04,3840.360107,3873.159912,3815.770020,3852.969971,4.414080e+09,^GSPC
2,2023-01-05,3839.739990,3839.739990,3802.419922,3808.100098,3.893450e+09,^GSPC
3,2023-01-06,3823.370117,3906.189941,3809.560059,3895.080078,3.923560e+09,^GSPC
4,2023-01-09,3910.820068,3950.570068,3890.419922,3892.090088,4.311770e+09,^GSPC
...,...,...,...,...,...,...,...
6748,2023-10-11,3897.689941,3927.010010,3816.580078,3836.889893,0.000000e+00,^JN0U.JO
6749,2023-10-12,3823.189941,3852.129883,3793.110107,3800.600098,0.000000e+00,^JN0U.JO
6750,2023-10-15,3807.610107,3858.659912,3782.330078,3851.270020,0.000000e+00,^JN0U.JO
6751,2023-10-16,3855.219971,3856.739990,3789.149902,3847.870117,0.000000e+00,^JN0U.JO


In [18]:
#Quick view at JN0UJO
market_indices_2023.loc[market_indices_2023["Index Ticker"] == "^JN0U.JO"]



,Date,Open,High,Low,Close,Volume,Index Ticker
6554,2023-01-02,4248.419922,4352.609863,4246.060059,4329.560059,0.0,^JN0U.JO
6555,2023-01-03,4330.350098,4410.689941,4324.819824,4361.839844,0.0,^JN0U.JO
6556,2023-01-04,4345.229980,4412.149902,4321.899902,4321.899902,0.0,^JN0U.JO
6557,2023-01-05,4343.850098,4434.589844,4340.450195,4428.609863,0.0,^JN0U.JO
6558,2023-01-08,4465.790039,4597.660156,4462.540039,4582.970215,0.0,^JN0U.JO
...,...,...,...,...,...,...,...
6748,2023-10-11,3897.689941,3927.010010,3816.580078,3836.889893,0.0,^JN0U.JO
6749,2023-10-12,3823.189941,3852.129883,3793.110107,3800.600098,0.0,^JN0U.JO
6750,2023-10-15,3807.610107,3858.659912,3782.330078,3851.270020,0.0,^JN0U.JO
6751,2023-10-16,3855.219971,3856.739990,3789.149902,3847.870117,0.0,^JN0U.JO


In [19]:
# market_indices_recent.to_csv("recent_market_indices.csv", index = False)

In [20]:
len(market_indices_2023["Index Ticker"].unique())

35

In [21]:
for index in world_indices.Symbol:
  tickerData = yf.Ticker(index)
  tickerDf = tickerData.history(period = "1d", start = "-1-1", end = "2023-10-17")
  tickerDf["Index Ticker"] = index
  market_indices_past.append(tickerDf)

market_indices_2023 = pd.concat(market_indices_2023, axis = 0) #Creating recent index dataframe
market_indices_2023.reset_index(inplace = True) #Fixing formatting, adding Date as a column

ValueError: time data '-1-1' does not match format '%Y-%m-%d'